# Importar bibliotecas

In [1]:
import os
import re
import csv
from datetime import datetime
import pytz
from shapely.geometry import shape, Point
import geopandas as gpd
from geopy.distance import distance
import pandas as pd
from h3 import h3
from geopy.distance import geodesic
import shapefile
from shapely.geometry import Point, shape
from shapely.ops import unary_union
import findspark
from pyspark.sql import SparkSession, Row
from pyspark.context import SparkContext
from pyspark.sql.functions import col, lit, concat_ws, collect_set
from pyspark.sql.types import TimestampType, StructField, StringType, IntegerType, StructType
from pyspark.sql.functions import date_format
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.regression import RandomForestRegressor
from pyspark.ml import Pipeline
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator
from sklearn.linear_model import LinearRegression
from pyspark.sql.functions import lit
from pyspark.ml.linalg import Vectors
from pyspark.sql.functions import col, udf
from pyspark.sql.types import DoubleType
from pyspark.ml.regression import DecisionTreeRegressor
from pyspark.sql import functions as F
import json
import folium
import io
import ast  # Para convertir la cadena de coordenadas a una tupla
from IPython.display import IFrame
from pyspark.sql.functions import year, month, dayofmonth, hour, dayofweek
from pyspark.sql.functions import when
import shutil

# Obtener rutas de los Datasets

In [15]:
# Obtener la ruta de la carpeta del notebook
notebook_folder = os.getcwd()
root_project = os.path.abspath(os.path.join(notebook_folder, '..'))
dataset_logs = os.path.abspath(os.path.join(root_project, 'Datos', 'Logs'))
trufi_datos = os.path.abspath(os.path.join(root_project, 'Datos', 'Registros de Trufi App'))
municipios_datos = os.path.abspath(os.path.join(root_project, 'Datos', 'Poligonos de Cochabamba','region_cochabamba_2018.geojson'))
clima_datos = os.path.abspath(os.path.join(root_project, 'Datos', 'Datos del clima','weather.csv'))
lagos_datos = os.path.abspath(os.path.join(root_project, 'Datos', 'Poligonos de Cochabamba','lagos.shx'))
h3_datos = os.path.abspath(os.path.join(root_project, 'Datos', 'Registros de Trufi App','id_index_h3.csv'))
map_cochabamba_file_path =  os.path.abspath(os.path.join(root_project, 'Datos', 'mapas','h3_map_cochabamba_.html'))
temporal_serie_data = os.path.abspath(os.path.join(root_project, 'Datos', 'Registros de Trufi App','temporal_serie_data'))
csv_file_path = os.path.join(trufi_datos, 'origen-destino.csv')

# Extraer solicitudes de logs a origen-destino.csv

In [3]:
# Definir la zona horaria de La Paz, Bolivia
la_paz_timezone = pytz.timezone('America/La_Paz')
def extract_route_info(log_line):
    # Expresión regular para extraer información específica de las solicitudes de rutas
    route_pattern_with_id = re.compile(r'GET /otp/plan\?fromPlace=([-0-9.]+)%2C([-0-9.]+)&toPlace=([-0-9.]+)%2C([-0-9.]+).*?Trufi/.*?/([a-f0-9-]+)')
    route_pattern_without_id = re.compile(r'GET /otp/plan\?fromPlace=([-0-9.]+)%2C([-0-9.]+)&toPlace=([-0-9.]+)%2C([-0-9.]+)')

    match_with_id = route_pattern_with_id.search(log_line)
    match_without_id = route_pattern_without_id.search(log_line)

    # Inicializar variables con valores predeterminados
    origin_latitude = origin_longitude = dest_latitude = dest_longitude = id_user = None

    try:
        if match_with_id:
            origin_latitude, origin_longitude, dest_latitude, dest_longitude, id_user = match_with_id.groups()
        elif match_without_id:
            origin_latitude, origin_longitude, dest_latitude, dest_longitude = match_without_id.groups()
            id_user = 'N/A'  # Asignar un valor predeterminado
        return origin_latitude, origin_longitude, dest_latitude, dest_longitude, id_user

    except Exception as e:
        print(f"Error al procesar la línea: {log_line}")
        print(f"Error: {e}")

    return None
def process_log_file(file_path):
    with open(file_path, 'r') as f:
        for line in f:
            if 'GET /otp/plan' in line:
                date_str = re.search(r'\[([^:]+:[^ ]+)', line).group(1)
                
                # Convertir la fecha a un objeto datetime y añadir la zona horaria de La Paz
                date_time = datetime.strptime(date_str, '%d/%b/%Y:%H:%M:%S').replace(tzinfo=pytz.utc).astimezone(la_paz_timezone)

                route_info = extract_route_info(line)
                if route_info:
                    origin_latitude, origin_longitude, dest_latitude, dest_longitude, id_user = route_info
                    
                    yield [date_time.strftime('%Y-%m-%d %H:%M:%S'),origin_latitude, origin_longitude, dest_latitude, dest_longitude, id_user]
                    
# Obtener la lista de archivos y ordenarlos según el prefijo numérico
file_pattern = re.compile(r'^\d{2}-')
files = os.listdir(dataset_logs)
log_files = [file for file in files if file_pattern.match(file)]
log_files.sort(key=lambda x: int(x.split('-')[0]))

header = ['date','origin_latitude', 'origin_longitude', 'destination_latitude', 'destination_longitude','userID']

with open(csv_file_path, 'w', newline='') as csvfile:
    csv_writer = csv.writer(csvfile)
    csv_writer.writerow(header)

    for log_file in log_files:
        file_path = os.path.join(dataset_logs, log_file)
        
        route_info_generator = process_log_file(file_path)
        
        # Escribir las líneas en el archivo CSV
        csv_writer.writerows(route_info_generator)

print(f"Las rutas que cumplen con el patrón se han guardado en {csv_file_path}.")


Las rutas que cumplen con el patrón se han guardado en D:\Trufiapp\GANS\Anexos\Datos\Registros de Trufi App\r_origen-destino.csv.


## Descartar solicitudes sin user ID 

In [4]:
def filter_requests_with_userid(csv_file_path):
    # Leer el archivo CSV
    df = pd.read_csv(csv_file_path, parse_dates=['date'])

    # Contar filas antes de la filtración
    total_rows_before = len(df)

    # Filtrar las solicitudes que tienen userID
    df_filtered = df[df['userID'].notnull()]

    # Contar filas después de la filtración
    total_rows_after = len(df_filtered)

    # Calcular cuántas filas fueron eliminadas
    rows_removed = total_rows_before - total_rows_after

    # Sobrescribir el archivo original con las solicitudes filtradas
    df_filtered.to_csv(csv_file_path, index=False)

    return total_rows_before, rows_removed

# Llamar a la función para filtrar las solicitudes
total_rows_before, rows_removed = filter_requests_with_userid(csv_file_path)

# Imprimir la cantidad de filas eliminadas
print(f"Se eliminaron {rows_removed} filas sin userID de {total_rows_before}.")

Se eliminaron 83 filas sin userID de 1577937.


## Cantidad de usuarios a dentro del rango de fecha evaluado

In [5]:
# Leer el archivo CSV en un DataFrame
df = pd.read_csv(csv_file_path, parse_dates=['date'])

# Obtener la cantidad de usuarios únicos
unique_users_count = df['userID'].nunique()

# Imprimir la cantidad de usuarios únicos
print(f"La cantidad de usuarios únicos es: {unique_users_count}")


La cantidad de usuarios únicos es: 99785


## Excluir puntos que esten dentro de lagos

In [6]:
df = pd.read_csv(csv_file_path, parse_dates=['date'])
df

,date,origin_latitude,origin_longitude,destination_latitude,destination_longitude,userID
0,2022-09-12 09:55:24,-17.399618,-66.161091,-17.382153,-66.165591,680c28ba-5008-4ffe-8026-3eaad4f832d1
1,2022-09-12 09:55:32,-17.379989,-66.167222,-17.390862,-66.159421,680c28ba-5008-4ffe-8026-3eaad4f832d1
2,2022-09-12 10:05:28,-17.390862,-66.159421,-17.379989,-66.167222,680c28ba-5008-4ffe-8026-3eaad4f832d1
3,2022-09-12 10:07:36,-17.373542,-66.165138,-17.383765,-66.159304,680c28ba-5008-4ffe-8026-3eaad4f832d1
4,2022-09-12 11:27:27,-17.383765,-66.159304,-17.373542,-66.165138,680c28ba-5008-4ffe-8026-3eaad4f832d1
...,...,...,...,...,...,...
1577849,2023-12-26 22:25:49,-17.388356,-66.197263,-17.377586,-66.058064,a4f158b9-1359-49a0-9d7f-d28f393727ee
1577850,2023-12-26 22:26:06,-17.418014,-66.160858,-17.401366,-66.182851,bb060471-bc27-4175-af11-301f0853bf33
1577851,2023-12-26 22:26:36,-17.412731,-66.159542,-17.383059,-66.159274,958041c4-c269-4603-9237-fb7b917f042b
1577852,2023-12-26 22:26:52,-17.412731,-66.159542,-17.369551,-66.174464,958041c4-c269-4603-9237-fb7b917f042b


In [7]:
# Leer el archivo CSV en un DataFrame
df = pd.read_csv(csv_file_path, parse_dates=['date'])

# Crear puntos para las columnas de latitud y longitud
df['origin_point'] = df.apply(lambda row: Point(row['origin_longitude'], row['origin_latitude']), axis=1)
df['destination_point'] = df.apply(lambda row: Point(row['destination_longitude'], row['destination_latitude']), axis=1)

# Crear un objeto multipolígono unión solo con los polígonos deseados
desired_objectIds = [3635, 3486, 3204, 2948]

shapes_reader = shapefile.Reader(lagos_datos)
shapes = shapes_reader.shapes()
records = shapes_reader.records()

# Obtener información de las formas deseadas
desired_shapes = [shape(shape_).buffer(0) for i, shape_ in enumerate(shapes) if records[i][0] in desired_objectIds]

# Filtrar DataFrame para quedarse solo con los puntos que están fuera de los polígonos deseados
df_filtered = df[~df['origin_point'].apply(lambda point: any(point.within(shape_) for shape_ in desired_shapes)) & ~df['destination_point'].apply(lambda point: any(point.within(shape_) for shape_ in desired_shapes))]
# Guardar el DataFrame filtrado en un nuevo archivo CSV sin las columnas origin_point y destination_point
df_filtered.drop(['origin_point', 'destination_point'], axis=1, inplace=True)


C:\Users\idaas\AppData\Local\Temp\ipykernel_9136\1986673735.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered.drop(['origin_point', 'destination_point'], axis=1, inplace=True)


In [8]:

# Contar líneas antes del filtrado
rows_before = len(df)

# Contar líneas después del filtrado
rows_after = len(df_filtered)

# Imprimir la cantidad de líneas antes y después del filtrado
print(f"Solicitudes antes del filtrado: {rows_before}")
print(f"Solicitudes después del filtrado: {rows_after}")

# Guardar el DataFrame filtrado en un nuevo archivo CSV
df_filtered.to_csv(csv_file_path, index=False)

print(f"Las solicitudes filtradas se han guardado en: {csv_file_path}")

Solicitudes antes del filtrado: 1577854
Solicitudes después del filtrado: 1577111
Las solicitudes filtradas se han guardado en: D:\Trufiapp\GANS\Anexos\Datos\Registros de Trufi App\r_origen-destino.csv


## Excluir solicitudes de rutas con una distancia menor a 500 mts entre origen y destino

In [9]:
# Leer el archivo CSV en un DataFrame
df = pd.read_csv(csv_file_path, parse_dates=['date'])

# Crear puntos para las columnas de latitud y longitud
df['origin_point'] = df.apply(lambda row: Point(row['origin_longitude'], row['origin_latitude']), axis=1)
df['destination_point'] = df.apply(lambda row: Point(row['destination_longitude'], row['destination_latitude']), axis=1)

# Calcular la distancia entre origen y destino en metros
df['distancia'] = df.apply(lambda row: distance((row['origin_latitude'], row['origin_longitude']),
                                               (row['destination_latitude'], row['destination_longitude'])).meters, axis=1)

# Filtrar DataFrame para quedarse solo con los puntos que tienen una distancia mayor a 500 metros
df_filtered = df[df['distancia'] > 500]

In [10]:
# Contar líneas antes del filtrado
rows_before = len(df)

# Contar líneas después del filtrado
rows_after = len(df_filtered)

# Imprimir la cantidad de líneas antes y después del filtrado
print(f"Solicitudes antes del filtrado: {rows_before}")
print(f"Solicitudes después del filtrado: {rows_after}")

# Guardar el DataFrame filtrado en un nuevo archivo CSV sin las columnas origin_point y destination_point
df_filtered.drop(['origin_point', 'destination_point'], axis=1, inplace=True)
df_filtered.to_csv(csv_file_path, index=False)

print(f"Las solicitudes filtradas se han guardado en: {csv_file_path}")

Solicitudes antes del filtrado: 1577111
Solicitudes después del filtrado: 1446500


C:\Users\idaas\AppData\Local\Temp\ipykernel_9136\2112296885.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered.drop(['origin_point', 'destination_point'], axis=1, inplace=True)


Las solicitudes filtradas se han guardado en: D:\Trufiapp\GANS\Anexos\Datos\Registros de Trufi App\r_origen-destino.csv


## Generar variables para municipios origen y destino

In [11]:
# Leer el archivo CSV en un DataFrame
df = pd.read_csv(csv_file_path, parse_dates=['date'])
    
# Agregar las columnas 'origin_municipio' y 'dest_municipio' con valores predeterminados
df['origin_municipio'] = 'externo'
df['dest_municipio'] = 'externo'

# Guardar el DataFrame actualizado en el archivo CSV
df.to_csv(csv_file_path, index=False)


In [12]:
cochabamba_data = gpd.read_file(municipios_datos)

In [13]:

def get_city_from_coords(coords):
    point = Point(coords)
    # Iterar sobre las características del GeoJSON
    for index, row in cochabamba_data.iterrows():
        city_geometry = row['geometry']
        # Verificar si el punto está dentro de la geometría de la ciudad
        if point.within(city_geometry):
            return row['CAPITAL']
    
    return 'externo'

# Agregar las columnas 'origin_municipio' y 'dest_municipio' al DataFrame
columns_to_add = ['origin_municipio', 'dest_municipio']

# Crear un DataFrame vacío con las nuevas columnas
df = pd.DataFrame(columns=columns_to_add)
# Abrir el archivo CSV para lectura y leer las columnas existentes
with open(csv_file_path, 'r') as csvfile:
    # Leer la primera línea para obtener los nombres de las columnas
    existing_columns = csvfile.readline().strip().split(',')

    # Crear un DataFrame vacío
    df = pd.DataFrame(columns=existing_columns)

    # Crear un archivo temporal para escribir las líneas actualizadas
    with open('temp_csvfile.csv', 'w', newline='') as temp_csvfile:
        # Escribir los nombres de las columnas en el archivo temporal
        temp_csvfile.write(','.join(existing_columns) + '\n')

        for i, line in enumerate(csvfile):
            # Leer una línea del archivo CSV
            row = pd.read_csv(io.StringIO(line), header=None, names=existing_columns).iloc[0]

            # Obtener nombres de municipios para origen y destino
            origin_municipio = get_city_from_coords((row['origin_longitude'], row['origin_latitude']))
            dest_municipio = get_city_from_coords((row['destination_longitude'], row['destination_latitude']))

            # Agregar los valores al DataFrame
            row['origin_municipio'] = origin_municipio
            row['dest_municipio'] = dest_municipio

            # Escribir la línea actualizada en el archivo temporal
            temp_csvfile.write(','.join(map(str, row.values)) + '\n')
shutil.move('temp_csvfile.csv', csv_file_path)

'D:\\Trufiapp\\GANS\\Anexos\\Datos\\Registros de Trufi App\\r_origen-destino.csv'

## Excluir solicitudes de rutas fuera de Cochabamba

In [17]:
# Leer el archivo CSV en un DataFrame
df = pd.read_csv(csv_file_path, parse_dates=['date'], encoding='latin1')
# Filtrar el DataFrame para excluir las solicitudes con origen o destino "externo"
df_filtered = df[(df['origin_municipio'] != 'externo') & (df['dest_municipio'] != 'externo')]

# Contar líneas antes del filtrado
rows_before = len(df)

# Contar líneas después del filtrado
rows_after = len(df_filtered)

# Imprimir la cantidad de líneas antes y después del filtrado
print(f"Solicitudes antes del filtrado: {rows_before}")
print(f"Solicitudes después del filtrado: {rows_after}")

# Guardar el DataFrame filtrado en un nuevo archivo CSV
df_filtered.to_csv(csv_file_path, index=False)

print(f"Las solicitudes filtradas se han guardado en: {csv_file_path}")

Solicitudes antes del filtrado: 1446251
Solicitudes después del filtrado: 1442497
Las solicitudes filtradas se han guardado en: D:\Trufiapp\GANS\Anexos\Datos\Registros de Trufi App\origen-destino.csv


## Generar datos sinteticos

In [18]:
# Agregar características de tiempo y distancia
df['hora'] = df['date'].dt.hour
df['dia_de_semana'] = df['date'].dt.dayofweek  # Lunes: 0, Domingo: 6
df['dia_de_mes'] = df['date'].dt.day
df['fin_de_semana'] = (df['date'].dt.weekday >= 5).astype(int)  # 1 si es fin de semana, 0 si no

# Mostrar el DataFrame resultante
print(df)

# Guardar el DataFrame filtrado en un nuevo archivo CSV
df.to_csv(csv_file_path, index=False)

print(f"Las solicitudes filtradas se han guardado en: {csv_file_path}")

                       date  origin_latitude  origin_longitude  \
0       2022-09-12 09:55:24       -17.399618        -66.161091   
1       2022-09-12 09:55:32       -17.379989        -66.167222   
2       2022-09-12 10:05:28       -17.390862        -66.159421   
3       2022-09-12 10:07:36       -17.373542        -66.165138   
4       2022-09-12 11:27:27       -17.383765        -66.159304   
...                     ...              ...               ...   
1446246 2023-12-26 22:25:49       -17.388356        -66.197263   
1446247 2023-12-26 22:26:06       -17.418014        -66.160858   
1446248 2023-12-26 22:26:36       -17.412731        -66.159542   
1446249 2023-12-26 22:26:52       -17.412731        -66.159542   
1446250 2023-12-26 22:29:54       -17.388356        -66.197263   

         destination_latitude  destination_longitude  \
0                  -17.382153             -66.165591   
1                  -17.390862             -66.159421   
2                  -17.379989          

## Integrar poligonos h3

In [19]:
# Función para obtener el índice H3 y las coordenadas
def get_h3_index_and_coords(latitude, longitude, resolution=7):
    h3_index = h3.geo_to_h3(latitude, longitude, resolution)
    coords = h3.h3_to_geo(h3_index)
    return h3_index, coords

# Leer el archivo CSV en un DataFrame
df = pd.read_csv(csv_file_path, parse_dates=['date'], encoding='latin1')

# Crear columnas para índices H3 de origen y destino
df['origin_h3_index'], df['origin_coords'] = zip(*df.apply(lambda row: get_h3_index_and_coords(row['origin_latitude'], row['origin_longitude']), axis=1))
df['dest_h3_index'], df['dest_coords'] = zip(*df.apply(lambda row: get_h3_index_and_coords(row['destination_latitude'], row['destination_longitude']), axis=1))

# Crear un diccionario que mapea cada índice H3 único a su propio ID único
h3_index_to_id = {h3_index: idx + 1 for idx, h3_index in enumerate(pd.concat([df['origin_h3_index'], df['dest_h3_index']]).unique())}

# Crear DataFrame con índices H3 únicos y sus IDs correspondientes y coordenadas
h3_id_df = pd.DataFrame(list(h3_index_to_id.items()), columns=['h3_index', 'id'])
h3_id_df[['latitude', 'longitude']] = pd.DataFrame(h3_id_df['h3_index'].apply(lambda h3_index: h3.h3_to_geo(h3_index)).to_list(), index=h3_id_df.index)

# Guardar DataFrame de índices H3 y sus IDs en un archivo CSV separado
h3_id_df.to_csv(h3_datos, index=False)

# Asignar los IDs únicos tanto para el origen como para el destino utilizando el diccionario
df['origin_id'] = df['origin_h3_index'].map(h3_index_to_id)
df['destination_id'] = df['dest_h3_index'].map(h3_index_to_id)

# Crear un nuevo DataFrame solo con las columnas necesarias
new_df = df[['date','userID','distancia','origin_municipio','dest_municipio','hora','dia_de_semana','dia_de_mes','fin_de_semana','origin_h3_index','dest_h3_index','origin_id','destination_id']]

new_df.to_csv(csv_file_path, index=False)

print(f"La información modificada se ha guardado en el archivo {csv_file_path}.")
max_origin_id = df['origin_id'].max()
max_destination_id = df['destination_id'].max()

print(f"El máximo ID de destinos en Cochabamba es: {max_destination_id}")

La información modificada se ha guardado en el archivo D:\Trufiapp\GANS\Anexos\Datos\Registros de Trufi App\origen-destino.csv.
El máximo ID de destinos en Cochabamba es: 1113


In [20]:
# Leer el archivo CSV en un DataFrame
df = pd.read_csv(csv_file_path, parse_dates=['date'], encoding='latin1')

# Obtener una lista de IDs únicos correspondientes a origin_id
unique_origin_ids = df['origin_id'].unique().tolist()
unique_destination_ids = df['destination_id'].unique().tolist()

# Imprimir la lista de IDs únicos
print("Lista de IDs únicos de origin_id:")
print(unique_origin_ids)
print("Lista de IDs únicos de destination_id:")
print(unique_destination_ids)

Lista de IDs únicos de origin_id:
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215

In [21]:
# Load the DataFrame with H3 indices and coordinates
h3_id_df = pd.read_csv(h3_datos)

# Calculate the average latitude and longitude
avg_lat = h3_id_df['latitude'].mean()
avg_lon = h3_id_df['longitude'].mean()

# Create a map centered at the average coordinates
m = folium.Map(location=[avg_lat, avg_lon], zoom_start=12)

# Set to keep track of printed hexagons
printed_hexagons = set()

h3_id_df.head()

,h3_index,id,latitude,longitude
0,878b2c8a1ffffff,1,-17.397907,-66.169306
1,878b2c8aeffffff,2,-17.375229,-66.167926
2,878b2c8a3ffffff,3,-17.387033,-66.148877
3,878b2c8a0ffffff,4,-17.409708,-66.150256
4,878b2c98affffff,5,-17.455051,-66.153015


In [23]:
# Load the DataFrame with H3 indices and coordinates
h3_id_df = pd.read_csv(h3_datos)
m = folium.Map(location=[-17.392695, -66.156850], zoom_start=12)

# Set to keep track of printed hexagons
printed_hexagons = set()

# Function to add a polygon to the map with a label
def add_polygon_with_label(m, hexagon, label):
    folium.Polygon(
        locations=hexagon,
        color='blue',
        fill=True,
        fill_color='blue',
        fill_opacity=0.4,
        tooltip=f'Label: {label}'  # Add tooltip with the label number
    ).add_to(m)
    printed_hexagons.add(label)  # Use 'label' as a unique identifier

# Iterate over the records
for idx, row in h3_id_df.iterrows():
    # Get the coordinates and convert them to a tuple
    coords = (row['latitude'], row['longitude'])
    hexagon_coords = h3.h3_to_geo_boundary(row['h3_index'])  # Get hexagon coordinates

    # Check if the hexagon has not been printed yet
    if row['id'] not in printed_hexagons:  # Use 'id' as a unique identifier
        add_polygon_with_label(m, hexagon_coords, row['id'])  # Use 'id' as a label

m.save(map_cochabamba_file_path)
# Imprimir los hexágonos impresos
print("Hexágonos impresos:", len(printed_hexagons))

Hexágonos impresos: 1113


# Convertir a set de datos temporales

In [24]:
findspark.init()
findspark.find()
sc = SparkContext.getOrCreate()
# Primera sesión de Spark
spark = SparkSession.builder \
    .appName("Trufi") \
    .config("spark.executor.memoryOverhead", "2g") \
    .config("spark.executor.memory", "3g") \
    .config("spark.driver.host", "localhost") \
    .config("spark.master", "local") \
    .getOrCreate()

# Configurar el número de particiones
spark.conf.set("spark.sql.shuffle.partitions", "4")
spark.sparkContext

<SparkContext master=local[*] appName=pyspark-shell>

## Contar cuantas solicitudes de origen por zona y hora existen
## we are replacing  Pickup_datetime_hourly by Travel_datetime_hourly

In [25]:
# DataFrame original
df = spark.read.csv(csv_file_path, header=True, inferSchema=True)

# Convert the timestamp to an hourly format 
df = df.withColumn("Travel_datetime_hourly", date_format(col("date").cast("timestamp"), "yyyy-MM-dd HH:00"))

# Create a trip count column
df = df.withColumn("Trip_count", lit(1))

# Group by pickup datetime, location ID, and aggregate the trip count
hourly_aggregated = df.groupby(['Travel_datetime_hourly', 'origin_id']).agg({'Trip_count': 'count'}).withColumnRenamed("count(Trip_count)", "Trips_count")

# Crear la columna única usando concat_ws
hourly_aggregated = hourly_aggregated.select(
    concat_ws('_', col("Travel_datetime_hourly"), col("origin_id")).alias("UniqueColumn"),
    col("Travel_datetime_hourly"),
    col("origin_id"),
    col("Trips_count")
)

hourly_aggregated.orderBy(col("Travel_datetime_hourly").desc())
# Count the number of rows in the aggregated DataFrame
print("Total rows in the aggregated DataFrame:", hourly_aggregated.count())


Total rows in the aggregated DataFrame: 209458


In [26]:
# DataFrame original
df = spark.read.csv(csv_file_path, header=True, inferSchema=True)
df.show()

+-------------------+--------------------+------------------+----------------+--------------+----+-------------+----------+-------------+---------------+---------------+---------+--------------+
|               date|              userID|         distancia|origin_municipio|dest_municipio|hora|dia_de_semana|dia_de_mes|fin_de_semana|origin_h3_index|  dest_h3_index|origin_id|destination_id|
+-------------------+--------------------+------------------+----------------+--------------+----+-------------+----------+-------------+---------------+---------------+---------+--------------+
|2022-09-12 09:55:24|680c28ba-5008-4ff...| 1991.214698938187|      Cochabamba|    Cochabamba|   9|            0|        12|            0|878b2c8a1ffffff|878b2c8aeffffff|        1|             2|
|2022-09-12 09:55:32|680c28ba-5008-4ff...|1461.2453834316614|      Cochabamba|    Cochabamba|   9|            0|        12|            0|878b2c8aeffffff|878b2c8a3ffffff|        2|             3|
|2022-09-12 10:05:28|680c

## we are replacing  OriginLocationID by LocationID

In [27]:
df = df.withColumn("LocationID", col("origin_id").cast("string"))
# Reemplaza "df" con tu DataFrame actual y "origin_id_string" con el nombre de tu nueva columna
df = df.withColumn("UniqueColumn", concat_ws('_', df["date"], df["LocationID"]))

# Seleccionar las columnas deseadas
df = df.select("UniqueColumn", "date", "LocationID","distancia")

# Mostrar algunas filas para verificar
df.show()
df.count()

+--------------------+-------------------+----------+------------------+
|        UniqueColumn|               date|LocationID|         distancia|
+--------------------+-------------------+----------+------------------+
|2022-09-12 09:55:...|2022-09-12 09:55:24|         1| 1991.214698938187|
|2022-09-12 09:55:...|2022-09-12 09:55:32|         2|1461.2453834316614|
|2022-09-12 10:05:...|2022-09-12 10:05:28|         3|1461.2453834316614|
|2022-09-12 10:07:...|2022-09-12 10:07:36|         2|1290.2219151819006|
|2022-09-12 11:27:...|2022-09-12 11:27:27|         3|1290.2219151819006|
|2022-09-12 11:27:...|2022-09-12 11:27:34|         2|1290.2219151819006|
|2022-09-12 18:26:...|2022-09-12 18:26:30|         4| 1894.633275882146|
|2022-09-12 18:46:...|2022-09-12 18:46:18|         3| 7318732.780424042|
|2022-09-12 18:54:...|2022-09-12 18:54:27|         3| 4277.174107755568|
|2022-09-12 18:54:...|2022-09-12 18:54:42|         3| 4277.174107755568|
|2022-09-12 19:12:...|2022-09-12 19:12:20|         

1446251

In [28]:
def generate_series(start, stop, interval):
    start = pd.Timestamp(start)
    stop = pd.Timestamp(stop)
    timestamps = pd.date_range(start, stop, freq=f'{interval}s')
    return pd.DataFrame({'timestamp': timestamps})

# Generar el DataFrame de marcas de tiempo
timestamp_df = generate_series("2022-09-12", "2023-12-27", 3600)  # Intervalo de 1 hora

timestamp_df.tail()

,timestamp
11300,2023-12-26 20:00:00
11301,2023-12-26 21:00:00
11302,2023-12-26 22:00:00
11303,2023-12-26 23:00:00
11304,2023-12-27 00:00:00


In [29]:
# Generar el DataFrame de marcas de tiempo con intervalo de 1 hora
timestamp_df = pd.date_range("2022-09-12", "2023-12-27", freq='1H').to_frame(name='timestamp')

# Aplicar el formato de fecha deseado
timestamp_df['hourly_timestamp'] = timestamp_df['timestamp'].dt.strftime('%Y-%m-%d %H:00')
# Eliminar la última fila del DataFrame
timestamp_df = timestamp_df.iloc[:-1]

# Verificar la longitud después de la eliminación
print(len(timestamp_df))
timestamp_df

11304


,timestamp,hourly_timestamp
2022-09-12 00:00:00,2022-09-12 00:00:00,2022-09-12 00:00
2022-09-12 01:00:00,2022-09-12 01:00:00,2022-09-12 01:00
2022-09-12 02:00:00,2022-09-12 02:00:00,2022-09-12 02:00
2022-09-12 03:00:00,2022-09-12 03:00:00,2022-09-12 03:00
2022-09-12 04:00:00,2022-09-12 04:00:00,2022-09-12 04:00
...,...,...
2023-12-26 19:00:00,2023-12-26 19:00:00,2023-12-26 19:00
2023-12-26 20:00:00,2023-12-26 20:00:00,2023-12-26 20:00
2023-12-26 21:00:00,2023-12-26 21:00:00,2023-12-26 21:00
2023-12-26 22:00:00,2023-12-26 22:00:00,2023-12-26 22:00


In [30]:
# Lee el archivo en un DataFrame de Spark
id_index_df = spark.read.csv(h3_datos, header=True, inferSchema=True)

# Encuentra los IDs de ubicación únicos del DataFrame original
Pick_up_LocationID = id_index_df.select('id').distinct()
Pick_up_LocationID = Pick_up_LocationID.orderBy("id")
Pick_up_LocationID = Pick_up_LocationID.withColumnRenamed("id", "LocationID")
# Convierte la columna 'LocationID' a tipo entero
Pick_up_LocationID = Pick_up_LocationID.withColumn("LocationID", Pick_up_LocationID["LocationID"].cast(IntegerType()))

# Convertir elementos de unique_origin_ids a enteros
unique_origin_ids_int = [int(id) for id in unique_origin_ids]

# Crear un DataFrame de Spark a partir de la lista unique_origin_ids_int
row_list = [Row(LocationID=id) for id in unique_origin_ids_int]
schema = ["LocationID"]
Pick_up_LocationID = spark.createDataFrame(row_list, schema=schema)

# Mostrar la cantidad de filas en el nuevo DataFrame
print("Cantidad de filas en Pick_up_LocationID:", Pick_up_LocationID.count())

Cantidad de filas en Pick_up_LocationID: 804


In [31]:
# Define una función para agregar la columna locationID al DataFrame de timestamp
def locations_timestamp_generator(dataframe, location_id):
    dataframe["LocationID"] = location_id
    return dataframe

# Define el esquema para el DataFrame final
data_schema = [StructField("timestamp", TimestampType(), True),
               StructField("hourly_timestamp", StringType(), True),
               StructField("LocationID", IntegerType(), True)]
final_struct = StructType(fields=data_schema)
# Crea un DataFrame vacío con las columnas requeridas
data_schema = ["timestamp", "hourly_timestamp", "LocationID"]
ID_plus_timestamp = pd.DataFrame(columns=data_schema)

In [32]:
# Lista para almacenar DataFrames temporales
frames = []

# Itera sobre los IDs de ubicación y agrega las filas al DataFrame final
for row in Pick_up_LocationID.collect():
    location_id = row["LocationID"]
    timestamp_df_copy = timestamp_df.copy()
    timestamp_df_copy = locations_timestamp_generator(timestamp_df_copy, location_id)
    frames.append(timestamp_df_copy)

# Concatena todos los DataFrames en la lista
ID_plus_timestamp = pd.concat(frames, ignore_index=True)

# Muestra el resultado
ID_plus_timestamp.count()

timestamp           9088416
hourly_timestamp    9088416
LocationID          9088416
dtype: int64

In [33]:
print("Total de filas en el DataFrame resultante:", len(ID_plus_timestamp))
print(ID_plus_timestamp.tail())

Total de filas en el DataFrame resultante: 9088416
                  timestamp  hourly_timestamp  LocationID
9088411 2023-12-26 19:00:00  2023-12-26 19:00         804
9088412 2023-12-26 20:00:00  2023-12-26 20:00         804
9088413 2023-12-26 21:00:00  2023-12-26 21:00         804
9088414 2023-12-26 22:00:00  2023-12-26 22:00         804
9088415 2023-12-26 23:00:00  2023-12-26 23:00         804


In [34]:
# Cambiar el locationID de entero a cadena
ID_plus_timestamp["locationID_string"] = ID_plus_timestamp["LocationID"].astype(str)

# Crear la columna única utilizando concat_ws
ID_plus_timestamp["UniqueColumn_G"] = ID_plus_timestamp["hourly_timestamp"] + "_" + ID_plus_timestamp["locationID_string"]

# Seleccionar las columnas relevantes
ID_plus_timestamp = ID_plus_timestamp[["UniqueColumn_G", "timestamp", "hourly_timestamp", "LocationID"]]

# Mostrar algunas filas para verificar
print(ID_plus_timestamp.head())
ID_plus_timestamp.count()

       UniqueColumn_G           timestamp  hourly_timestamp  LocationID
0  2022-09-12 00:00_1 2022-09-12 00:00:00  2022-09-12 00:00           1
1  2022-09-12 01:00_1 2022-09-12 01:00:00  2022-09-12 01:00           1
2  2022-09-12 02:00_1 2022-09-12 02:00:00  2022-09-12 02:00           1
3  2022-09-12 03:00_1 2022-09-12 03:00:00  2022-09-12 03:00           1
4  2022-09-12 04:00_1 2022-09-12 04:00:00  2022-09-12 04:00           1


UniqueColumn_G      9088416
timestamp           9088416
hourly_timestamp    9088416
LocationID          9088416
dtype: int64

In [36]:
id_plus_timestamp = spark.createDataFrame(ID_plus_timestamp.copy())
hourly_aggregated.head()

----------------------------------------
Exception occurred during processing of request from ('127.0.0.1', 51756)
ERROR:root:Exception while sending command.
Traceback (most recent call last):
  File "C:\Users\idaas\anaconda3\envs\pyspark-env\lib\site-packages\py4j\clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
  File "C:\Users\idaas\anaconda3\envs\pyspark-env\lib\socket.py", line 704, in readinto
    return self._sock.recv_into(b)
ConnectionResetError: [WinError 10054] An existing connection was forcibly closed by the remote host

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\idaas\anaconda3\envs\pyspark-env\lib\site-packages\py4j\java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "C:\Users\idaas\anaconda3\envs\pyspark-env\lib\site-packages\py4j\clientserver.py", line 539, in send_command
    raise Py4JNetworkE

Py4JError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.readRDDFromFile

In [ ]:
# Cambiar el ID de ubicación de entero a cadena
id_plus_timestamp = id_plus_timestamp.withColumn("locationID_string", col("LocationID").cast("int"))

# Crear la columna única usando concat_ws
id_plus_timestamp = id_plus_timestamp.select(
    concat_ws('_', col("hourly_timestamp"), col("locationID_string")).alias("UniqueColumn_G"),
    col("timestamp"),
    col("hourly_timestamp"),
    col("LocationID"),
    col("hourly_timestamp")
)

# Realizar la unión de ambos DataFrames usando la columna única
pick_aggregated = hourly_aggregated.join(id_plus_timestamp, hourly_aggregated["UniqueColumn"] == id_plus_timestamp["UniqueColumn_G"], "right")
# Mostrar algunas filas del resultado
pick_aggregated.head()

In [ ]:
# Seleccionar solo las columnas deseadas
columnas_deseadas = ['timestamp', 'hourly_timestamp', 'LocationID', 'Trips_count', 'UniqueColumn_G']
pick_aggregated_o = pick_aggregated.select(*columnas_deseadas)

# Ordenar por las columnas en el orden deseado
pick_aggregated_o = pick_aggregated_o.orderBy(*[col(column) for column in columnas_deseadas])
# Reemplazar los valores nulos en las columnas especificadas
pick_aggregated_o = pick_aggregated_o.na.fill(0, "Trips_count")
pick_aggregated_o = pick_aggregated_o.na.fill(0, "LocationID")

# Cambiar el nombre de Trips_count a origin_request_count
pick_aggregated_o = pick_aggregated_o.withColumnRenamed("Trips_count", "origin_request_count")

# Mostrar el DataFrame ordenado
pick_aggregated_o.show()

In [ ]:
# Especifica la ruta del archivo CSV en el sistema de archivos local
csv_local_path = "file:///" + temporal_serie_data

# Guardar el DataFrame en formato CSV en el sistema de archivos local
pick_aggregated_o.write.mode("overwrite").option("header", "true").csv(csv_local_path)

In [ ]:
# Lee el archivo CSV en un DataFrame
df = spark.read.csv("file:///" + temporal_serie_data, header=True, inferSchema=True)

# Muestra el esquema del DataFrame
df.printSchema()

# Muestra las primeras filas del DataFrame
df.show()

## Contar cuantos destinos por zona y hora existen

In [ ]:
# Read the CSV file into a DataFrame
df = spark.read.csv(csv_file_path, header=True, inferSchema=True)

# Convert the timestamp to an hourly format
df = df.withColumn("Travel_datetime_hourly", date_format(col("date").cast("timestamp"), "yyyy-MM-dd HH:00"))

# Create a trip count column
df = df.withColumn("Trip_count", lit(1))

# Group by pickup datetime, location ID, and aggregate the trip count
hourly_aggregated = df.groupby(['Travel_datetime_hourly', 'destination_id']).agg({'Trip_count': 'count'}).withColumnRenamed("count(Trip_count)", "Trips_count")

# Crear la columna única usando concat_ws
hourly_aggregated = hourly_aggregated.select(
    concat_ws('_', col("Travel_datetime_hourly"), col("destination_id")).alias("UniqueColumn"),
    col("Travel_datetime_hourly"),
    col("destination_id"),
    col("Trips_count")
)

# Show the resulting DataFrame
hourly_aggregated.show()

# Count the number of rows in the aggregated DataFrame
print("Total rows in the aggregated DataFrame:", hourly_aggregated.count())


In [ ]:
# Realiza un left join de los DataFrames usando las columnas UniqueColumn y UniqueColumn_G
merged_df = pick_aggregated_o.join(hourly_aggregated, pick_aggregated_o["UniqueColumn_G"] == hourly_aggregated["UniqueColumn"], "left_outer")

# Selecciona las columnas deseadas en el DataFrame final
final_merged = merged_df.select(
    pick_aggregated_o["UniqueColumn_G"],
    pick_aggregated_o["hourly_timestamp"],
    pick_aggregated_o["LocationID"],
    hourly_aggregated["Trips_count"].alias("destination_request_count"),
    pick_aggregated_o["origin_request_count"]
)

# Muestra el DataFrame final
final_merged.show()
final_merged.count()

In [ ]:
final_merged = final_merged.na.fill(0, "destination_request_count")
final_merged.orderBy(col("hourly_timestamp").desc()).show(10)
final_merged.show(1000)

In [ ]:
# Create Year column
final_merged = final_merged.withColumn('anio', year(final_merged['hourly_timestamp']))

# Create Month column
final_merged = final_merged.withColumn('mes', month(final_merged['hourly_timestamp']))

# Create Day of Month column
final_merged = final_merged.withColumn('dia_de_mes', dayofmonth(final_merged['hourly_timestamp']))

# Create Hour column
final_merged = final_merged.withColumn('hora', hour(final_merged['hourly_timestamp']))

# Create Day of Week column
final_merged = final_merged.withColumn("dia_de_semana", dayofweek(col("hourly_timestamp")))

# Crear la columna isWeekend
final_merged = final_merged.withColumn("fin_de_semana", when((col("dia_de_semana") == 1) | (col("dia_de_semana") == 7), 1).otherwise(0))

# Show the resulting DataFrame
final_merged.count()


In [ ]:

# Especifica la ruta del archivo CSV en el sistema de archivos local
csv_local_path = "file:///" + temporal_serie_data

# Guardar el DataFrame en formato CSV en el sistema de archivos local
final_merged.write.mode("overwrite").option("header", "true").csv(csv_local_path)


In [ ]:
spark.stop()